# Code combining mouse and keyboard

In [27]:

import mouse
import keyboard            
import pickle
import pandas as pd
import pyautogui as s


file = s.prompt('Enter filename for recording')


mouse_events = []


mouse.hook(mouse_events.append)
keyboard.start_recording()

keyboard.wait("Esc")

mouse.unhook(mouse_events.append)
keyboard_events = keyboard.stop_recording()


# put mouse results (click only) to list
list = []

for i in range(len(mouse_events)):
    if mouse_events[i][0] == 'down':
        # pick up previus coordinate
        #row = i - 1
        #x = mouse_events[row][0]
        #y = mouse_events[row][1]     
        #list.append([x,y])
        
        for chk in range(1,5):
        #    # check previous 5 rows until you dont see 'down' or 'up' key
            row = i - chk
            if mouse_events[row][0] not in ('down','up'):
                 x = mouse_events[row][0]
                 y = mouse_events[row][1]     
                 z = mouse_events[row][2]
                 list.append(['Mouse Click Event',x,y,z])
                 break   
                
# put keyboard results to list
string = []
stroke = []
finalword = []
ctrl = []

for m in range(len(keyboard_events)):
    mystring = str(keyboard_events[m])
    letter = mystring[14:15]
    if 'down' in mystring:
        if 'backspace' in mystring:
            #print (string)
            string.pop()
            #print (string)
            #list.append(['Keyboard Event','backspace'])
        elif 'space' in mystring:
             #list.append(['Keyboard Event','space'])
             string.append(' ')
        elif 'enter' in mystring:
               
            if len(string) != 0 :
                finalword = ''.join(string) 
                list.append(['Keyboard Write',finalword])
                list.append(['Keyboard Event','enter'])
                string = []
            
            #finalword = []
            #string.append(' Enter ')
        elif 'tab' in mystring:
            #list.append(['Keyboard Event','enter'])
            #string.append(' Tab ')
            if len(string) != 0 :
                finalword = ''.join(string) 
                list.append(['Keyboard Write',finalword])
                list.append(['Keyboard Event','tab'])
                string = []
            
        elif 'shift' in mystring:
            # shift means nothing for this logic as it is just uppercase or lowercase
            letter = ''
            
        elif 'ctrl' in mystring:
            # flag you are in control mode
            # no need to append strings and they should be blank
            ctrl = 'active'
            string = []
            #list.append(['Keyboard Ctrl',mystring])
            
        elif 'up' in mystring and ctrl != 'active':
            list.append(['Keyboard Ctrl','up'])
            
        elif 'right' in mystring and ctrl != 'active':
            list.append(['Keyboard Ctrl','right'])
            
        elif 'left' in mystring and ctrl != 'active':
            list.append(['Keyboard Ctrl','left'])
        
        elif 'esc' in mystring:
            #list.append(['Ignore','esc'])
            #print (mystring)
            if len(string) != 0 :
                finalword = ''.join(string) 
                list.append(['Keyboard Write',finalword])
                list.append(['Keyboard Event','enter'])
                string = []
        elif ctrl == 'active':
            
            
            if 'v down' in mystring:
                list.append(['Keyboard Ctrl','ctrl v'])
                ctrl = ''
                string = []
            elif 'c down' in mystring:
                list.append(['Keyboard Ctrl','ctrl c'])
                ctrl = ''
                string = []
            else:
                list.append(['Keyboard Ctrl',mystring])
                string = []
       
        else:
            string.append(letter)
            
            
# add duration to the list

for index, i in enumerate(list):
   try :
    if index != 0:
        now = list[index][3]
        before = list[index-1][3]
        duration = now - before
        list[index].append(duration)
    else:
        list[index].append(0)
   except:
        break
        
        
# tidy up the sequence between mouse and keyboards into a list

import pyautogui as s
import time

tmp = []
newlist = []
#count = 0

for index, i in enumerate(list):
    if 'Mouse' in i[0]:             # mouse in string 
        time = i[4]                 # time column
        if time < 2 and tmp == []:
            newlist.append(i)
        else:
            # if time > 2 seconds, move them into tmp list
            tmp.append(i)
    elif 'Keyboard Write' in i[0]:
        newlist.append(i)
    elif 'Keyboard Event' in i[0]:
        newlist.append(i)
        
        
        # first check if it is a tab event
        if i[0][1] == 'tab':
            # add the next two events to the list
            newlist.append(i+1)
            newlist.append(i+2)
        
       
        
        else:
        # after keyboard events will bring back those mouse events
        # from tmp list after the keyboards.
        # logic will loop until the next mouse event > 2 seconds
        # at the moment assumption is typing will be more than 2 seconds.
            for i in range(len(tmp)):

                if i == 0:     # first event, straight back to the main list
                    count = 0
                    newlist.append(tmp[count])
                    count += 1
                    #tmp.pop(i)
                    #i = i - 1
                elif tmp[i][4] < 2:      # if less than 2 seconds, keep adding to main list
                    newlist.append(tmp[count])
                    count += 1
                    #tmp.pop(i)
                    #i = i -1
                elif tmp[i][4] > 2:      # when the next keyboard even found (> 2 seconds), exit the loop
                    break
        
# if no keyboard events, length of new list will be shorter than the original list
# hence just use the original list

if len(newlist) != len(list):
    newlist = list

# put to dataframes
x1 = pd.DataFrame(list)
x2 = pd.DataFrame(newlist)
x1.columns = ['Event','X','Y','Time','Duration']
x2.columns = ['Event','X','Y','Time','Duration']
print ('Initial List \n',x1,'\n\n','New list \n',x2)


# save the recording    

try :
    
     # save the recording    
    with open(file + '.txt', 'wb') as fh:
       pickle.dump(newlist, fh)

    # raw list recording   
    with open(file + '_list' + '.txt', 'wb') as fh:
       pickle.dump(list, fh)
   
    # raw full mouse recording    
    with open(file + '_mouse' + '.txt', 'wb') as fh:
       pickle.dump(mouse_events, fh)

    # raw full keyboard recording
    with open(file + '_keyboard' + '.txt', 'wb') as fh:
       pickle.dump(keyboard_events, fh)

except :
    print ('file does not exist')




   

Initial List 
                 Event     X    Y          Time  Duration
0   Mouse Click Event   823    0  1.637668e+09  0.000000
1   Mouse Click Event  1196  516  1.637668e+09  1.529832
2   Mouse Click Event   338  324  1.637668e+09  3.437724
3   Mouse Click Event   458  298  1.637668e+09  1.196871
4   Mouse Click Event   545  373  1.637668e+09  2.394329
5   Mouse Click Event   496  475  1.637668e+09  2.102937
6   Mouse Click Event   677  375  1.637668e+09  1.189356
7   Mouse Click Event   639  470  1.637668e+09  1.291093
8   Mouse Click Event   823  383  1.637668e+09  1.181977
9   Mouse Click Event   790  473  1.637668e+09  1.152319
10  Mouse Click Event   959  370  1.637668e+09  1.144334
11  Mouse Click Event   877  474  1.637668e+09  1.931053
12  Mouse Click Event  1096  378  1.637668e+09  1.630504
13  Mouse Click Event  1044  472  1.637668e+09  1.430698 

 New list 
                 Event     X    Y          Time  Duration
0   Mouse Click Event   823    0  1.637668e+09  0.000000
1 

In [29]:
s.confirm('This displays text and has an OK and Cancel button.')

'Cancel'

In [28]:
# replay the click        
import pyautogui as s
import time

# open the text file where you recorded previous events

file = s.prompt('Filename to open?')
file = file + '.txt'

pickle_off = open (file, "rb")
newlist = pickle.load(pickle_off)

for i in range(len(newlist)):
    if 'Mouse' in newlist[i][0]:
        x = newlist[i][1]
        y = newlist[i][2]
        s.moveTo(x,y,0.5)
        s.click(clicks = 2, button='left', interval = 0.5)
        time.sleep(0.9)
    elif 'Write' in newlist[i][0]:
        #break
        #s.press('enter')
        s.write(newlist[i][1],0.3)
    elif 'tab' in newlist[i][1]:
        s.press('tab')
        time.sleep(0.9)
    elif 'enter' in newlist[i][1]:
        s.press('enter')
        time.sleep(0.9)
        
    # new codes for keyboards
    elif 'Ctrl' in newlist[i][0]:
         if 'left' in newlist[i][1]:
            s.hotkey('press','left')
            
         if 'right' in newlist[i][1]:
            s.hotkey('press','right')
            
         if 'up' in newlist[i][1]:
            s.hotkey('press','up')
            
         if 'down' in newlist[i][1]:
            s.hotkey('press','down')
            
         if 'ctrl c' in newlist[i][1]:
            s.hotkey('ctrl','c')
            
         if 'ctrl v' in newlist[i][1]:
            s.hotkey('ctrl','v')
            
            
            
            
            
            
            
            
            

# Once recorded, you can add parameters to the list

1. Define the parameter in param
2. specific the location of the index tochange variable
3. store the new variable under x
4. put that x value to the list

Re-run the replay

# Once recorded, you can add parameters to the list

1. Define the parameter in param
2. specific the location of the index tochange variable
3. store the new variable under x
4. put that x value to the list

Re-run the replay

In [122]:

import pickle

# open the text file where you recorded previous events

file = s.prompt('Filename to open?')
file = file + '.txt'

pickle_off = open (file, "rb")
modified = pickle.load(pickle_off)

modified

[['Mouse Click Event', 595, 14, 1637538135.3287408, 0],
 ['Mouse Click Event', 518, 527, 1637538136.5132012, 1.1844604015350342],
 ['Mouse Click Event', 149, 373, 1637538138.3052173, 1.7920160293579102],
 ['Mouse Click Event', 658, 363, 1637538140.329437, 2.0242197513580322],
 ['Mouse Click Event', 960, 135, 1637538141.752526, 1.4230890274047852],
 ['Mouse Click Event', 326, 186, 1637538143.5772655, 1.8247394561767578],
 ['Mouse Click Event', 331, 281, 1637538153.9213681, 10.344102621078491],
 ['Keyboard Write', 'CCAS3cur1ty'],
 ['Keyboard Event', 'enter']]

In [11]:
# to remove the last event, if you want to remove specific index type the number inside the bracket
modified.pop()

# inserting to a specific index of additional code
modified[7] = (['Keyboard Event','enter'])

# adding to the last row
modified.append(['Keyboard Event','enter'])

# mergin one list to another, e.g. previous file called old
modified.append(old[0])

# adding parameters
param = '2020-06-30'
tochange = modified[3][1]

x = str(tochange) + ' ' + str(param)
modified[3][1] = x

print (x)


['Mouse Click Event', 788, 20, 1637407689.5779638, 0]

In [123]:
modified.pop(0)

['Mouse Click Event', 595, 14, 1637538135.3287408, 0]

In [124]:
# after modified, save the text file again   
with open(file, 'wb') as fh:
   pickle.dump(modified, fh)


# Play around

In [8]:
# looping basic

for i in range(6):
    print (i)
    if i == 1:
        print ('found')
        break
    else:
        print ('greater than 4')

0
greater than 4
1
found


In [10]:

# basic while loop


import time
found = 0
start = time.time()
    
while found == 0:
    end = time.time()
    duration = end - start
    
    if duration < 5:
        found = 0
    else:
        found = 1
    
    
    